In [1]:
#!/usr/bin/env/python3

import requests, json, time, schedule, functools, threading
from datetime import datetime, timedelta 
from pymongo import MongoClient
from config import params

# Reserve Function
def getSched1():
          
    def routeList():
        client = MongoClient('mongodb://localhost:27017')
        db = client['guru_db2']
        collection = db['routes']
        routes = list(collection.find({},{"RouteID":1, "_id":0}))
        client.close()
        return routes    
          
    def getOneSched(routeID,date):
        schedurl = 'https://api.wmata.com/Bus.svc/json/jRouteSchedule?'
        response = requests.get(schedurl + "RouteID=" + routeID + "&Date=" + date , params=params).json()
        client = MongoClient('mongodb://localhost:27017')
        db = client['guru_db2']
        collection = db['sched1']
        acqTimeStamp = datetime.now().isoformat()[:-7]
        if len(response['Direction0']) != 0:
            for trip in response['Direction0']:
                trip['AcqTimeStamp'] = acqTimeStamp
                collection.insert_one(trip)  

        if len(response['Direction1']) != 0:
            for trip in response['Direction1']:
                trip['AcqTimeStamp'] = acqTimeStamp
                collection.insert_one(trip) 
        client.close()
        time.sleep(0.2)
        print(f"{routeID} :: {date} :: {len(response['Direction0'])+len(response['Direction1'])}")
          
    routes = routeList()
    dates = ["2019-08-01"]
    for date in dates:
        for route in routes:
            routeID = route['RouteID']
            getOneSched(routeID,date)
              
              
# Implemented Function             
def getSched2():
          
    def routeList():
        client = MongoClient('mongodb://localhost:27017')
        db = client['guru_db2']
        collection = db['routes']
        routes = list(collection.find({},{"RouteID":1, "_id":0}))
        client.close()
        return routes    
          
    def getOneSched(routeID,date):
        schedurl = 'https://api.wmata.com/Bus.svc/json/jRouteSchedule?'
        response = requests.get(schedurl + "RouteID=" + routeID, params=params).json()
        client = MongoClient('mongodb://localhost:27017')
        db = client['guru_db2']
        collection = db['sched']
        acqTimeStamp = datetime.now().isoformat()[:-7]
        response['acqTimeStamp'] = acqTimeStamp
        collection.insert_one(response)
        client.close()
        print(routeID)
        time.sleep(0.2)
                      
    routes = routeList()
    for date in dates:
        for route in routes:
            routeID = route['RouteID']
            getOneSched(routeID)